# Aqui inicia mi sufrimiento :(

# PROYECTO FINAL


## Sistemas Distribuidos con PySpark

Este proyecto implementa lo que se ha visto en clases en una serie de ejercicios a las cuales llamo un **sistema de recomendación de libros** utilizando:
- **100 libros** más descargados de Project Gutenberg (Omitiendo 2 ya que estos denegaban el acceso...)
- **PySpark** para procesamiento distribuido
- **TF-IDF** para análisis de texto
- **Similitud coseno** para encontrar libros similares
- Y las enseñanzas del profe

## CELDA 1: Configuración de Rutas

**¿Qué hace?**
Configura las rutas del proyecto para que Python pueda encontrar nuestros archivos de utilidades (`src/utils.py`).

**Explicación técnica:**
- `sys.path.append()` añade directorios donde Python buscará módulos <details> <summary> ¿Por que? </summary> (Antes tuvimos problemas con que el programa que no encontraba la ruta de SRC)</details>


- `..` = directorio padre (raíz del proyecto)
- `../src` = carpeta de código fuente<details>
    <summary> ¿Que hay ahi? </summary> 
    Ahi adentro se encuentran los archivos python donde uno descarga los libros (download_books.py) y otro donde se encarga de limpiar estos mismos como quitar los headers o footers para que no extorben en el analisis (utils.py)
</details>


In [1]:
import sys
import os

# Agregar rutas del proyecto
sys.path.append(os.path.abspath(".."))  # raíz del proyecto
sys.path.append(os.path.abspath("../src"))  # carpeta src

# Verificar rutas
print("Rutas configuradas:")
for path in sys.path[-3:]:
    print(f"  - {path}")

Rutas configuradas:
  - /home/arturo/venv/lib/python3.12/site-packages
  - /home/arturo/project_gutenberg
  - /home/arturo/project_gutenberg/src


## CELDA 2: Descargar Recursos de NLTK

**¿Qué hace?**
Descarga las herramientas necesarias de NLTK para procesar texto en inglés.

**Lo que descarga:**
- `punkt` y `punkt_tab` → Separa texto en palabras (tokenización)<details><summary>En otras palabras</summary>Esto divide un texto grande en piezas más pequeñas llamadas tokens.</details>

- `stopwords` → Lista de palabras comunes que no aportan significado ("the", "a", "is") <details>
  <summary> ¿De donde sacamos esta herramienta? </summary>
  Esta herramienta o mas bien libreria fue recomendada por el profesor para facilitarnos la detecciòn de stopwords
</details>

In [2]:
import nltk

print("Descargando recursos de NLTK...")
nltk.download('punkt_tab', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
print("✓ NLTK configurado")

Descargando recursos de NLTK...
✓ NLTK configurado


## CELDA 3: Inicializar Apache Spark

**¿Qué hace?**
Arranca el motor de Spark que procesará nuestros datos de forma distribuida.

**Configuración:**
- `driver.memory: 6g` → Memoria del coordinador (6 GB)<details><summary>Mas detalles</summary>Memoria para el driver.
    - Este driver.memory recibe las instrucciones, reparte el trabajo y al final recoge los resultados.
    - Los 6 GB significa que la máquina virtual va a reservar 6 GB solo para este coordinador.
    - Esto procesara los 100 libros
</details>

- `executor.memory: 4g` → Memoria de los trabajadores (4 GB)
- `local[*]` → Usa todos los cores de tu CPU

\
**¿Por qué Spark?**
- Para procesar 100 libros (~75 MB) de forma eficiente y paralela.<details>
    <summary> Y... </summary> 
    ...porque el profe nos lo pidio
</details>


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ProyectoFinal_RecomendacionLibros") \
    .master("local[*]") \
    .config("spark.driver.memory", "6g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()

# Reducir logs
spark.sparkContext.setLogLevel("WARN")

print("✓ Spark inicializado")
print(f"  Version: {spark.version}")
print(f"  Master: {spark.sparkContext.master}")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/07 23:28:24 WARN Utils: Your hostname, arturo-VirtualBox, resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/12/07 23:28:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/07 23:28:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


✓ Spark inicializado
  Version: 4.0.1
  Master: local[*]


## CELDA 4: Importar Funciones de Utilidad



**¿Qué hace?**\
Carga las funciones que creamos para procesar los libros desde la carpeta src al archivo utils.py

**Funciones importadas:**
- `read_txt()` → Lee un archivo .txt sin importar el tipo de encoding, evitando errores por acentos o caracteres raros.
- `strip_gutenberg_headers()` → Quita las licencias, advertencias y texto extra que agregan los libros.
- `preprocess_text()` → Limpia el texto aplicando estos filtros:
    - minúsculas
    - tokenizar
    - quitar stopwords
    - quitar puntuación<br>
- `load_all_books()` → Busca todos los .txt de la carpeta y:\
  Es la función que carga todos los libros del proyecto.
    - los lee
    - los limpia
    - los convierte en una lista/diccionario de libros procesados

In [4]:
from src.utils import (
    read_txt,
    strip_gutenberg_headers,
    preprocess_text,
    load_all_books
)

print("✓ Utilidades importadas")

✓ Utilidades importadas


## CELDA 5 y 6: Cargar 100 Libros

**¿Qué hace?**
Lee los 100 lirbos .txt desde la carpeta `data/` y los preprocesa.

**¿En que consiste el proceso?**
1. Lee el archivo
2. Elimina headers/footers
3. Convierte a minúsculas
4. Separa en palabras (tokens)
5. Elimina stopwords ("the", "a", etc.)
6. Elimina puntuación y números

**Resultado:**
Una lista con 100 libros, cada uno con:
- ID del libro (usando los ID originales de la pagina)
- Nombre del archivo
- Texto completo
- Lista de palabras limpias (tokens)

In [5]:
# Desmarcar el comentario en caso de extrema emergencia (No tener los libros)
# python3 src/download_books.py

In [6]:
data_dir = "../data"
books = load_all_books(data_dir, max_books=100)

print(f"\n📊 Resumen de carga:")
print(f"   Total de libros: {len(books)}")
if books:
    ejemplo = books[0]
    print(f"   Ejemplo - ID: {ejemplo[0]}, Tokens: {len(ejemplo[3])}")


📚 Cargando 100 libros desde ../data/
  ✓ Procesados 10/100 libros
  ✓ Procesados 20/100 libros
  ✓ Procesados 30/100 libros
  ✓ Procesados 40/100 libros
  ✓ Procesados 50/100 libros
  ✓ Procesados 60/100 libros
  ✓ Procesados 70/100 libros
  ✓ Procesados 80/100 libros
  ✓ Procesados 90/100 libros
  ✓ Procesados 100/100 libros
✅ Total de libros cargados exitosamente: 100


📊 Resumen de carga:
   Total de libros: 100
   Ejemplo - ID: 100, Tokens: 498519


## CELDA 7: Crear DataFrame de Spark


**¿Qué hace?**
Convierte nuestra lista de libros en un **DataFrame de Spark** (como una especie de tabla de Excel gigante).

**Columnas del DataFrame:**
- `book_id` → ID único del libro (ej: "84", "1342")
- `title` → Nombre del archivo (ej: "84.txt")
- `text` → Texto completo del libro
- `tokens` → Lista de palabras procesadas

**¿Por qué un DataFrame?**
Spark puede procesar DataFrames de forma paralela y distribuida.
<details><summary>Comentario</summary> Creemos que no se ve las columnas de "text" y "tokens" porque estamos usando una maquina virutal para correr este codigo y la verdad a este punto en el que estoy hubiera preferido instalar linux pero en la laptop, casi no la uso</details>
<br/>


In [7]:
from pyspark.sql.types import ArrayType, StringType

print("Creando DataFrame de Spark...")

df = spark.createDataFrame(
    [(b[0], b[1], b[2], b[3]) for b in books],
    schema=["book_id", "title", "text", "tokens"]
)

print(f"✓ DataFrame creado con {df.count()} documentos\n")
df.select("book_id", "title").show(5, truncate=False)


Creando DataFrame de Spark...


25/12/07 23:29:49 WARN TaskSetManager: Stage 0 contains a task of very large size (74586 KiB). The maximum recommended task size is 1000 KiB.
                                                                                

✓ DataFrame creado con 100 documentos



25/12/07 23:31:38 WARN TaskSetManager: Stage 3 contains a task of very large size (74586 KiB). The maximum recommended task size is 1000 KiB.
                                                                                

+-------+---------+
|book_id|title    |
+-------+---------+
|100    |100.txt  |
|1023   |1023.txt |
|10554  |10554.txt|
|1080   |1080.txt |
|11     |11.txt   |
+-------+---------+
only showing top 5 rows


## CELDA 8: Renombrar Columna (IGNORAR)

**¿Qué hace?**\
Renombra la columna `tokens` a `tokens_clean` para claridad.

**¿Por qué?**\
Al inicio lo dejamos para ver si nos daba los tokens y TODO el texto del libro pero al final lo dejamos asi porque nos dio miedo que se rompiera el codigo y tener que esperar otros 15 minutos a que corriera esto en la maquina virutal, asi que lo dejamos como algo para ignorar.


In [8]:
from pyspark.sql.functions import col

df = df.withColumnRenamed("tokens", "tokens_clean")

print("✓ Tokens ya procesados (stopwords removidas con NLTK)")
print("\nEjemplo de tokens limpios:")
df.select("book_id", "tokens_clean").show(3, truncate=60)

✓ Tokens ya procesados (stopwords removidas con NLTK)

Ejemplo de tokens limpios:


25/12/07 23:32:38 WARN TaskSetManager: Stage 4 contains a task of very large size (74586 KiB). The maximum recommended task size is 1000 KiB.
                                                                                

+-------+------------------------------------------------------------+
|book_id|                                                tokens_clean|
+-------+------------------------------------------------------------+
|    100|[complete, works, william, shakespeare, william, shakespe...|
|   1023|[bleak, house, charles, dickens, contents, preface, chanc...|
|  10554|[right, ho, jeeves, wodehouse, raymond, needham, affectio...|
+-------+------------------------------------------------------------+
only showing top 3 rows


## CELDA 9: Crear Vocabulario (CountVectorizer)

**¿Qué hace?**\
Crea un **vocabulario** con las 2000 palabras más importantes de todos los libros.

**Proceso:**
1. Cuenta cuántas veces aparece cada palabra en cada libro
2. Selecciona las 2000 palabras más frecuentes
3. Filtra palabras que aparecen en menos de 2 libros (muy raras)

**Resultado:**\
Cada libro se representa como un vector de 2000 números (frecuencias de palabras).
<details>
  <summary>Detalles del codigo</summary>
  <p>Parámetros:</p>
  <ul>
    <li>VOCAB_SIZE=2000: limita el vocabulario a las 2000 palabras más frecuentes. Evita vectores enormes.</li>
    <li>MIN_DF=2: elimina palabras que aparecen en menos de 2 documentos (filtra ruido / typos).</li>
  </ul>
</details>





In [9]:
from pyspark.ml.feature import CountVectorizer

print("Creando vocabulario con CountVectorizer...")

VOCAB_SIZE = 2000
MIN_DF = 2  # Palabra que debe aparecer en al menos 2 documentos

cv = CountVectorizer(
    inputCol="tokens_clean",
    outputCol="raw_features",
    vocabSize=VOCAB_SIZE,
    minDF=MIN_DF
)

cv_model = cv.fit(df)
df = cv_model.transform(df)

actual_vocab_size = len(cv_model.vocabulary)
print(f"✓ Vocabulario creado")
print(f"  Tamaño del vocabulario: {actual_vocab_size} palabras")
print(f"  Top 10 palabras: {cv_model.vocabulary[:10]}")

Creando vocabulario con CountVectorizer...


25/12/07 23:33:44 WARN TaskSetManager: Stage 5 contains a task of very large size (74586 KiB). The maximum recommended task size is 1000 KiB.
                                                                                

✓ Vocabulario creado
  Tamaño del vocabulario: 2000 palabras
  Top 10 palabras: ['one', 'said', 'would', 'man', 'could', 'upon', 'time', 'good', 'like', 'well']


## CELDA 10: Calcular TF-IDF

**¿Qué hace?**\
Calcula **TF-IDF** (Term Frequency - Inverse Document Frequency) para cada palabra.

**¿Qué es TF-IDF?**\
Un número que indica el **peso** o **importancia** en una palabra para un libro específico.

**Fórmula simple:**
- **TF** (frecuencia): ¿Cuántas veces aparece en ESTE libro?
- **IDF** (rareza): ¿Qué tan rara es en TODOS los libros?
- **TF-IDF** = TF × IDF

**Ejemplo:**
- "elizabeth" aparece mucho en Frankenstein → TF alto
- "elizabeth" NO aparece en otros libros → IDF alto
- **TF-IDF de "elizabeth"** = ALTO (palabra característica)

**¿Al final que se obtiene de todo esto?:**\
Cada libro tiene un vector TF-IDF que representa su contenido único.


In [10]:
from pyspark.ml.feature import IDF

print("\nCalculando TF-IDF...")

idf = IDF(inputCol="raw_features", outputCol="tfidf")
idf_model = idf.fit(df)
df = idf_model.transform(df)

print("✓ TF-IDF calculado")


Calculando TF-IDF...


25/12/07 23:35:31 WARN TaskSetManager: Stage 9 contains a task of very large size (74586 KiB). The maximum recommended task size is 1000 KiB.
                                                                                

✓ TF-IDF calculado


## CELDA 11: Normalizar Vectores


**¿Qué hace?**\
Normaliza los vectores TF-IDF para que todos tengan la misma "longitud" matemática.<details> <summary>Ejemplo</summary> Es como comparar la composición de dos bebidas sin importar el tamaño del vaso.</details>

**¿Por qué normalizar?**
- Libros largos tienen números más grandes
- Queremos comparar **proporcionalmente**, no por tamaño
- Después de normalizar, los valores están entre 0 y 1


**Resultado:**\
Vectores `tfidf_norm` con valores entre 0 y 1.


In [11]:
from pyspark.ml.feature import Normalizer

print("Normalizando vectores TF-IDF...")

normalizer = Normalizer(inputCol="tfidf", outputCol="tfidf_norm", p=2.0)
df = normalizer.transform(df)

print("✓ Vectores normalizados")
print("\nDataFrame final:")
df.select("book_id", "title", "tfidf_norm").show(4, truncate=60)

Normalizando vectores TF-IDF...
✓ Vectores normalizados

DataFrame final:


25/12/07 23:38:29 WARN TaskSetManager: Stage 10 contains a task of very large size (74586 KiB). The maximum recommended task size is 1000 KiB.
                                                                                

+-------+---------+------------------------------------------------------------+
|book_id|    title|                                                  tfidf_norm|
+-------+---------+------------------------------------------------------------+
|    100|  100.txt|(2000,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,...|
|   1023| 1023.txt|(2000,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,...|
|  10554|10554.txt|(2000,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,...|
|   1080| 1080.txt|(2000,[0,1,2,3,4,5,6,7,8,9,10,11,12,14,15,16,17,18,19,20,...|
+-------+---------+------------------------------------------------------------+
only showing top 4 rows


## CELDA 12: Crear Matriz de Similitud

**¿Qué hace?**
Crea una **matriz 100×100** que compara cada libro con todos los demás.

**Proceso:**
1. Convierte vectores de Spark a NumPy (arrays de Python)
2. Calcula el **producto punto** entre todos los pares de libros
3. El resultado es la **similitud coseno**


In [12]:
import numpy as np

print("Creando matriz de similitud...")

# Recolectar datos
rows = df.select("book_id", "title", "tfidf_norm").collect()

# Preparar estructuras de datos
book_ids = [r["book_id"] for r in rows]
id_to_title = {r["book_id"]: r["title"] for r in rows}

# Convertir vectores a numpy
print("  → Convirtiendo vectores a numpy...")
vectors = np.array([r["tfidf_norm"].toArray() for r in rows])

# Calcular matriz de similitud (producto punto = cosine similarity)
print("  → Calculando similitudes...")
sim_matrix = np.dot(vectors, vectors.T)

# Crear mapeos
index_to_id = {i: book_ids[i] for i in range(len(book_ids))}
id_to_index = {book_ids[i]: i for i in range(len(book_ids))}

print(f"✓ Matriz de similitud creada: {sim_matrix.shape}")
print(f"  Rango de similitudes: [{sim_matrix.min():.4f}, {sim_matrix.max():.4f}]")


Creando matriz de similitud...


25/12/07 23:39:22 WARN TaskSetManager: Stage 11 contains a task of very large size (74586 KiB). The maximum recommended task size is 1000 KiB.
                                                                                

  → Convirtiendo vectores a numpy...
  → Calculando similitudes...
✓ Matriz de similitud creada: (100, 100)
  Rango de similitudes: [0.0000, 1.0000]


##  CELDA 13 y 14: Recomendar libros



**¿Qué hace?**\
Ingresamos un libro, seguido de ello se encuentra los N libros más similares (Por default y para evitar que la memoria muera pusimos 5).

**Pasos:**
1. Busca el libro en la matriz de similitud
2. Obtiene sus similitudes con todos los demás libros
3. Ordena de mayor a menor similitud
4. Retorna los top N


In [13]:
def recomendar_libros(libro_id, N=5):
    libro_id = str(libro_id)
    
    if libro_id not in id_to_index:
        raise ValueError(f"❌ Libro '{libro_id}' no encontrado")
    
    idx = id_to_index[libro_id]
    similarities = sim_matrix[idx]

    pairs = [
        (index_to_id[i], float(similarities[i]))
        for i in range(len(similarities))
        if i != idx
    ]

    pairs_sorted = sorted(pairs, key=lambda x: x[1], reverse=True)
    
    results = [
        (bid, id_to_title[bid], score)
        for bid, score in pairs_sorted[:N]
    ]
    
    return results


In [14]:
recomendar_libros(100, N=5)

[('8800', '8800.txt', 0.7089103711909744),
 ('26', '26.txt', 0.6649206732336214),
 ('779', '779.txt', 0.6484633338043468),
 ('3296', '3296.txt', 0.6249226014288817),
 ('2680', '2680.txt', 0.5586967707233)]

## CELDA 15 y 16: Función Palabras Importantes (NORMALIZADA)


**¿Qué hace?**\
Encuentra las M palabras más características de un libro.

**Proceso:**
1. Obtiene el vector TF-IDF normalizado del libro
2. Ordena las palabras por su score TF-IDF
3. Retorna las top M palabras

**¿Por qué usa `tfidf_norm`?**
Los valores están entre 0 y 1, más fáciles de interpretar como porcentajes.

**Entrada:**
- `libro_id` = "84"
- `M` = 5

**Salida:**
```
elizabeth → 0.64 (64% de importancia)
feelings  → 0.15 (15% de importancia)
henry     → 0.12 (12% de importancia)
```

In [15]:
from pyspark.sql.functions import col

def palabras_importantes(documento_id, M=10):
    """
    🎯 FUNCIÓN PRINCIPAL #2 (VERSIÓN NORMALIZADA)
    
    Regresa M palabras que describen un documento.
    Usa vectores normalizados (valores entre 0 y 1).
    
    Parámetros:
        documento_id: ID del documento (book_id)
        M: Número de palabras a retornar
    
    Retorna:
        Lista de tuplas (palabra, score_normalizado)
    """
    # Buscar documento - CAMBIO AQUÍ: tfidf_norm en lugar de tfidf
    row = df.filter(col("book_id") == documento_id).select("tfidf_norm").collect()
    
    if not row:
        raise ValueError(f"❌ Documento '{documento_id}' no encontrado")
    
    # Obtener vector TF-IDF normalizado
    tfidf_vector = row[0]["tfidf_norm"]
    vocab = cv_model.vocabulary
    
    # Ordenar por score
    items = list(zip(tfidf_vector.indices, tfidf_vector.values))
    items_sorted = sorted(items, key=lambda x: x[1], reverse=True)
    
    # Convertir a palabras
    top_words = [
        (vocab[idx], float(val))
        for idx, val in items_sorted[:M]
    ]
    
    return top_words


print("✓ Función palabras_importantes() definida (versión normalizada)")

✓ Función palabras_importantes() definida (versión normalizada)


In [16]:
palabras_importantes(84, M=5)

25/12/07 23:42:40 WARN TaskSetManager: Stage 12 contains a task of very large size (74586 KiB). The maximum recommended task size is 1000 KiB.
                                                                                

[('elizabeth', 0.6382942221796127),
 ('feelings', 0.14750844941878943),
 ('henry', 0.12106935995588752),
 ('misery', 0.11192872548646718),
 ('william', 0.10978409395308954)]

## INPUTS

### CELDA 17: Catálogo Completo
Muestra los 100 libros disponibles con sus IDs.

In [17]:
# CELDA RESULTADOS 1: Mostrar catálogo completo
print("\n" + "="*80)
print(" CATÁLOGO COMPLETO DE LIBROS")
print("="*80)
print(f"\nTotal: {len(book_ids)} libros\n")

for i, bid in enumerate(book_ids, 1):
    print(f"{i:3}. ID: {bid:6} → {id_to_title[bid]}")

print("\n💡 Copia el ID del libro que te interese para las siguientes celdas")



 CATÁLOGO COMPLETO DE LIBROS

Total: 100 libros

  1. ID: 100    → 100.txt
  2. ID: 1023   → 1023.txt
  3. ID: 10554  → 10554.txt
  4. ID: 1080   → 1080.txt
  5. ID: 11     → 11.txt
  6. ID: 110    → 110.txt
  7. ID: 1184   → 1184.txt
  8. ID: 120    → 120.txt
  9. ID: 1232   → 1232.txt
 10. ID: 1259   → 1259.txt
 11. ID: 1260   → 1260.txt
 12. ID: 1342   → 1342.txt
 13. ID: 135    → 135.txt
 14. ID: 1399   → 1399.txt
 15. ID: 1400   → 1400.txt
 16. ID: 145    → 145.txt
 17. ID: 1497   → 1497.txt
 18. ID: 1513   → 1513.txt
 19. ID: 161    → 161.txt
 20. ID: 16119  → 16119.txt
 21. ID: 16328  → 16328.txt
 22. ID: 16389  → 16389.txt
 23. ID: 1661   → 1661.txt
 24. ID: 17135  → 17135.txt
 25. ID: 17199  → 17199.txt
 26. ID: 1727   → 1727.txt
 27. ID: 174    → 174.txt
 28. ID: 17450  → 17450.txt
 29. ID: 18035  → 18035.txt
 30. ID: 1952   → 1952.txt
 31. ID: 1998   → 1998.txt
 32. ID: 205    → 205.txt
 33. ID: 2160   → 2160.txt
 34. ID: 24029  → 24029.txt
 35. ID: 244    → 244.txt
 36. ID

### CELDA 18: Recomendador con INPUT
El usuario ingresa:
- ID del libro
- Cuántas recomendaciones quiere
→ Sistema devuelve libros similares

In [18]:
libro_id = input("\n📖 Ingresa el ID del libro: ").strip()
n_recs = int(input("🔢 ¿Cuántas recomendaciones?: ") or "5")

if libro_id in id_to_title:
    print(f"\n{'='*80}")
    print(f"📖 LIBRO BASE: {id_to_title[libro_id]}")
    print(f"{'='*80}\n")
    
    recomendaciones = recomendar_libros(libro_id, N=n_recs)
    
    for i, (bid, title, score) in enumerate(recomendaciones, 1):
        print(f"{i}. [{score:.4f}] {title} (ID: {bid})")
    
    print(f"\n✅ {n_recs} recomendaciones generadas")
else:
    print(f"❌ Libro '{libro_id}' no encontrado")



📖 Ingresa el ID del libro:  84
🔢 ¿Cuántas recomendaciones?:  6



📖 LIBRO BASE: 84.txt

1. [0.5918] 1342.txt (ID: 1342)
2. [0.3883] 36965.txt (ID: 36965)
3. [0.3776] 4085.txt (ID: 4085)
4. [0.3518] 205.txt (ID: 205)
5. [0.3493] 36034.txt (ID: 36034)
6. [0.3236] 5197.txt (ID: 5197)

✅ 6 recomendaciones generadas


### CELDA 19: Palabras Características con INPUT
El usuario ingresa:
- ID del libro
- Cuántas palabras quiere
→ Sistema muestra palabras clave con barras visuales

In [19]:
libro_id = input("\n📖 Ingresa el ID del libro: ").strip()
m_palabras = int(input("🔢 ¿Cuántas palabras?: ") or "10")

if libro_id in id_to_title:
    print(f"\n{'='*80}")
    print(f"📖 LIBRO: {id_to_title[libro_id]}")
    print(f"{'='*80}\n")
    
    palabras = palabras_importantes(libro_id, M=m_palabras)
    
    for i, (palabra, score) in enumerate(palabras, 1):
        # Barra visual
        bar = "█" * int((score / palabras[0][1]) * 30)
        print(f"{i:2}. {palabra:20} │ {score:7.2f} │ {bar}")
    
    print(f"\n✅ Top {m_palabras} palabras generadas")
else:
    print(f"❌ Libro '{libro_id}' no encontrado")


📖 Ingresa el ID del libro:  84
🔢 ¿Cuántas palabras?:  8



📖 LIBRO: 84.txt



25/12/07 23:46:23 WARN TaskSetManager: Stage 13 contains a task of very large size (74586 KiB). The maximum recommended task size is 1000 KiB.
                                                                                

 1. elizabeth            │    0.64 │ ██████████████████████████████
 2. feelings             │    0.15 │ ██████
 3. henry                │    0.12 │ █████
 4. misery               │    0.11 │ █████
 5. william              │    0.11 │ █████
 6. miserable            │    0.10 │ ████
 7. horror               │    0.10 │ ████
 8. beheld               │    0.10 │ ████

✅ Top 8 palabras generadas


### CELDA 20: Análisis Completo con INPUT
El usuario ingresa:
- ID del libro
- Número de recomendaciones
- Número de palabras
→ Sistema hace análisis completo

In [20]:

libro_id = input("\n📖 ID del libro: ").strip()
n_recs = int(input("🔢 Recomendaciones (default 5): ") or "5")
m_palabras = int(input("🔢 Palabras clave (default 10): ") or "10")

if libro_id in id_to_title:
    analizar_libro(libro_id, n_recomendaciones=n_recs, m_palabras=m_palabras)
else:
    print(f"❌ Libro '{libro_id}' no encontrado")


📖 ID del libro:  84
🔢 Recomendaciones (default 5):  
🔢 Palabras clave (default 10):  


NameError: name 'analizar_libro' is not defined